<a href="https://colab.research.google.com/github/KhyatiMahendru/MusicGenreClassification/blob/master/MusicGenreClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Genre Classification: Rock v/s Hip-Hop

In [0]:
#importing required dependencies and libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
uploaded = files.upload()

Saving echonest-metrics.json to echonest-metrics.json
Saving fma-rock-vs-hiphop.csv to fma-rock-vs-hiphop.csv


# Collecting dataset

In [0]:
# meta-data file
tracks = pd.read_csv(io.BytesIO(uploaded['fma-rock-vs-hiphop.csv']))

# metrics file
echonest_metrics = pd.read_json(io.BytesIO(uploaded['echonest-metrics.json']))

#merging to get final data-set
echo_tracks = pd.merge(echonest_metrics, tracks[["track_id", "genre_top"]], on = "track_id")
echo_tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4802 entries, 0 to 4801
Data columns (total 10 columns):
track_id            4802 non-null int64
acousticness        4802 non-null float64
danceability        4802 non-null float64
energy              4802 non-null float64
instrumentalness    4802 non-null float64
liveness            4802 non-null float64
speechiness         4802 non-null float64
tempo               4802 non-null float64
valence             4802 non-null float64
genre_top           4802 non-null object
dtypes: float64(8), int64(1), object(1)
memory usage: 412.7+ KB


# Correlation Analysis

In [0]:
corr_metrics = echo_tracks.corr()
corr_metrics.style.background_gradient()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,1,-0.372282,0.0494541,0.140703,-0.275623,0.0482307,-0.0269951,-0.0253918,0.0100698
acousticness,-0.372282,1,-0.0289537,-0.281619,0.19478,-0.0199914,0.072204,-0.0263097,-0.0138406
danceability,0.0494541,-0.0289537,1,-0.242032,-0.255217,-0.106584,0.276206,-0.242089,0.473165
energy,0.140703,-0.281619,-0.242032,1,0.0282377,0.113331,-0.109983,0.195227,0.0386027
instrumentalness,-0.275623,0.19478,-0.255217,0.0282377,1,-0.0910218,-0.366762,0.022215,-0.219967
liveness,0.0482307,-0.0199914,-0.106584,0.113331,-0.0910218,1,0.0411725,0.00273169,-0.0450931
speechiness,-0.0269951,0.072204,0.276206,-0.109983,-0.366762,0.0411725,1,0.00824055,0.149894
tempo,-0.0253918,-0.0263097,-0.242089,0.195227,0.022215,0.00273169,0.00824055,1,0.0522212
valence,0.0100698,-0.0138406,0.473165,0.0386027,-0.219967,-0.0450931,0.149894,0.0522212,1


# Creating training and testing datasets

In [0]:
X = echo_tracks.drop(['track_id','genre_top'], axis = 1)
Y = echo_tracks.loc[:, 'genre_top']
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.33, random_state = 33, stratify = Y)

### Normalising the independent variables

In [0]:
scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
testX = scaler.transform(testX)
echo_tracks.head()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,genre_top
0,2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,Hip-Hop
1,3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,Hip-Hop
2,341,0.977282,0.468808,0.134975,0.687700,0.105381,0.073124,119.646,0.430707,Rock
3,46204,0.953349,0.498525,0.552503,0.924391,0.684914,0.028885,78.958,0.430448,Rock
4,46205,0.613229,0.500320,0.487992,0.936811,0.637750,0.030327,112.667,0.824749,Rock


# Classification

# 1. Logistic Regression

In [0]:
clf1 = LogisticRegression(random_state = 0, max_iter = 1000, solver = 'lbfgs')
clf1.fit(trainX, trainY)
pred1 = clf1.predict(testX)
print("Accuracy Score using Logistic Regression = ", accuracy_score(testY, pred1))

Accuracy Score using Logistic Regression =  0.9022082018927445


# 2. Decision Tree Classifier

In [0]:
clf2 = DecisionTreeClassifier()
clf2.fit(trainX, trainY)
pred2 = clf2.predict(testX)
print("Accuracy Score using Decision Tree Classifier = ", accuracy_score(testY, pred2))

Accuracy Score using Decision Tree Classifier =  0.8769716088328076


# 3. k-Nearest Neighbors

In [0]:
clf3 = KNeighborsClassifier(n_neighbors = 10)
clf3.fit(trainX, trainY)
pred3 = clf3.predict(testX)
print("Accuracy Score using kNN (k = 10) = ", accuracy_score(testY, pred3))

Accuracy Score using kNN (k = 10) =  0.9141955835962146


# 4. Naive Bayes Classifier

In [0]:
clf4 = GaussianNB()
clf4.fit(trainX, trainY)
pred4 = clf4.predict(testX)
print("Accuracy Score using Naive Baiyes Classifier = ", accuracy_score(testY, pred4))

Accuracy Score using Naive Baiyes Classifier =  0.8958990536277602


# 5. Support Vector Machine Classifier

In [0]:
clf3 = SVC(gamma='scale', decision_function_shape='ovo')
clf3.fit(trainX, trainY)
pred3 = clf3.predict(testX)
print("Accuracy Score using Support Vector Machine Classifier = ", accuracy_score(testY, pred3))

Accuracy Score using Support Vector Machine Classifier =  0.9211356466876972


# 6. Multi Layer Perceptron

In [0]:
# setting seed for replicating results
np.random.seed(7)

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(trainY)
encoded_Y = encoder.transform(trainY)
encoded_test_Y = encoder.transform(testY)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_test_y = np_utils.to_categorical(encoded_test_Y)

In [0]:
# building the model
model = Sequential()
model.add(Dense(256, input_shape=(8,))) #8 features
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(256))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))

model.add(Dense(2))                     #2 classes
model.add(Activation('softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
#Training the model, simultanoeusly validating
model.fit(trainX, dummy_y, epochs=800, batch_size = 32, validation_data=(testX, dummy_test_y))

Instructions for updating:
Use tf.cast instead.
Train on 3217 samples, validate on 1585 samples
Epoch 1/800
3217/3217 [==============================] - 1s 311us/step - loss: 0.4078 - acc: 0.8241 - val_loss: 0.2517 - val_acc: 0.9066
Epoch 2/800
3217/3217 [==============================] - 0s 105us/step - loss: 0.2882 - acc: 0.8865 - val_loss: 0.2298 - val_acc: 0.9117
Epoch 3/800
3217/3217 [==============================] - 0s 109us/step - loss: 0.2630 - acc: 0.8962 - val_loss: 0.2127 - val_acc: 0.9155
Epoch 4/800
3217/3217 [==============================] - 0s 107us/step - loss: 0.2451 - acc: 0.9033 - val_loss: 0.2172 - val_acc: 0.9148
Epoch 5/800
3217/3217 [==============================] - 0s 106us/step - loss: 0.2467 - acc: 0.9024 - val_loss: 0.2054 - val_acc: 0.9205
Epoch 6/800
3217/3217 [==============================] - 0s 104us/step - loss: 0.2324 - acc: 0.9077 - val_loss: 0.2026 - val_acc: 0.9243
Epoch 7/800
3217/3217 [==============================] - 0s 103us/step - loss: 0.2

In [0]:
import math
testScore = model.evaluate(testX, dummy_test_y, verbose = 0)
print('Model loss = ', testScore[0])
print('Model accuracy = ', testScore[1])

Model loss =  0.49057236880722105
Model accuracy =  0.9173501577663121
